&emsp;&emsp;&emsp;&emsp;&emsp;
&emsp;&emsp;&emsp;&emsp;&emsp;
&emsp;&emsp;&emsp;&emsp;&emsp;
&emsp;&emsp;&emsp;&emsp;&emsp;
&emsp;&emsp;&emsp;&emsp;&emsp;
&emsp;&emsp;&ensp;
[Home Page](../START_HERE.ipynb)

[Previous Notebook](03-Cudf_Exercise.ipynb)
&emsp;&emsp;&emsp;&emsp;&emsp;
&emsp;&emsp;&emsp;&emsp;&emsp;
&emsp;&emsp;&emsp;&emsp;&emsp;
&emsp;&emsp;&emsp;&emsp;&emsp;
[1](01-Intro_to_cuDF.ipynb)
[2](02-Intro_to_cuDF_UDFs.ipynb)
[3](03-Cudf_Exercise.ipynb)
[4]


# Applying CuDF: Workbook

Welcome to fourth cuDF tutorial notebook! This is a practical example that utilizes cuDF and cuPy, geared primarily for new users. The purpose of this tutorial is to introduce new users to a data science processing pipeline using RAPIDS on real life datasets. We will be working on a data science problem: US Accidents Prediction. This is a countrywide car accident dataset, which covers 49 states of the USA. The accident data are collected from February 2016 to June 2020, using two APIs that provide streaming traffic incident (or event) data. These APIs broadcast traffic data captured by a variety of entities, such as the US and state departments of transportation, law enforcement agencies, traffic cameras, and traffic sensors within the road-networks. Currently, there are about 3.5 million accident records in this dataset. 


## What should I do?

Given below is a complete data science preprocessing pipeline for the dataset using Pandas and Numpy libraries. Using the methods and techniques from the previous notebooks, you have to convert this pipeline to a a RAPIDS implementation, using CuDF and CuPy. Don't forget to time your code cells and compare the performance with this original code, to understand why we are using RAPIDS. If you get stuck in the middle, feel free to refer to this sample solution. 

## Here is the list of exercises in the lab where you need to modify code:
- <a href='#ex1'>Exercise 1</a><br> Loading the dataset from a csv file and store in a CuDF dataframe.
- <a href='#ex2'>Exercise 2</a><br> Creating kernel functions to run the given function optimally on a GPU.


The first step is downloading the dataset and putting it in the data directory, for using in this tutorial.
Download the dataset here, and place it in (host/data) folder. Now we will import the necessary libraries.

In [1]:
import os
import cudf
import numpy as np
import cupy as cp
import math
np.random.seed(12)

<a id='ex1'></a>

First we need to load the dataset from the csv into CuDF dataframes, for the preprocessing steps. If you need help, refer to the Getting Data In and Out module from this [notebook](01-Intro_to_cuDF.ipynb/).

In [ ]:
#Modify the code in this cell

# Use cudf to read csv
%time df =                        
print(df)


First we will analyse the data and observe patterns that can help us process the data better for feeding to the machine learning algorithms in the future. By using the describe, we will generate the descriptive statistics for all the columns. Descriptive statistics include those that summarize the central tendency, dispersion and shape of a dataset’s distribution, excluding NaN values.

In [3]:
df.describe()

,TMC,Severity,Start_Lat,Start_Lng,End_Lat,End_Lng,Distance(mi),Number,Temperature(F),Wind_Chill(F),Humidity(%),Pressure(in),Visibility(mi),Wind_Speed(mph),Precipitation(in)
count,2.478818e+06,3.513617e+06,3.513617e+06,3.513617e+06,1.034799e+06,1.034799e+06,3.513617e+06,1.250753e+06,3.447885e+06,1.645368e+06,3.443930e+06,3.457735e+06,3.437761e+06,3.059008e+06,1.487743e+06
mean,2.080226e+02,2.339929e+00,3.654194e+01,-9.579151e+01,3.755758e+01,-1.004560e+02,2.816170e-01,5.975383e+03,6.193512e+01,5.355730e+01,6.511427e+01,2.974463e+01,9.122644e+00,8.219025e+00,1.598300e-02
std,2.076627e+01,5.521930e-01,4.883520e+00,1.736877e+01,4.861215e+00,1.852879e+01,1.550134e+00,1.496624e+04,1.862106e+01,2.377334e+01,2.275558e+01,8.319760e-01,2.885879e+00,5.262847e+00,1.928260e-01
min,2.000000e+02,1.000000e+00,2.455527e+01,-1.246238e+02,2.457011e+01,-1.244978e+02,0.000000e+00,0.000000e+00,-8.900000e+01,-8.900000e+01,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,2.010000e+02,2.000000e+00,3.363784e+01,-1.174418e+02,3.399477e+01,-1.183440e+02,0.000000e+00,8.640000e+02,5.000000e+01,3.570000e+01,4.800000e+01,2.973000e+01,1.000000e+01,5.000000e+00,0.000000e+00
50%,2.010000e+02,2.000000e+00,3.591687e+01,-9.102601e+01,3.779736e+01,-9.703438e+01,0.000000e+00,2.798000e+03,6.400000e+01,5.700000e+01,6.700000e+01,2.995000e+01,1.000000e+01,7.000000e+00,0.000000e+00
75%,2.010000e+02,3.000000e+00,4.032217e+01,-8.093299e+01,4.105139e+01,-8.210168e+01,1.000000e-02,7.098000e+03,7.590000e+01,7.200000e+01,8.400000e+01,3.009000e+01,1.000000e+01,1.150000e+01,0.000000e+00
max,4.060000e+02,4.000000e+00,4.900220e+01,-6.711317e+01,4.907500e+01,-6.710924e+01,3.336300e+02,9.999997e+06,1.706000e+02,1.150000e+02,1.000000e+02,5.774000e+01,1.400000e+02,9.840000e+02,2.500000e+01


We will check the size of the dataset that is to be processed using the len function.

In [4]:
len(df)

3513617

You will notice that the dataset has 3513616 rows and takes quite a lot of time to read from the file. As we go ahead with the preprocessing, computations will require more time to execute, and that's where the RAPIDS comes to the rescue!

Now we use the info function to check the datatype of all the columns in the dataset.

In [5]:
df.info()

<class 'cudf.core.dataframe.DataFrame'>
RangeIndex: 3513617 entries, 0 to 3513616
Data columns (total 49 columns):
 #   Column                 Dtype
---  ------                 -----
 0   ID                     object
 1   Source                 object
 2   TMC                    float64
 3   Severity               int64
 4   Start_Time             object
 5   End_Time               object
 6   Start_Lat              float64
 7   Start_Lng              float64
 8   End_Lat                float64
 9   End_Lng                float64
 10  Distance(mi)           float64
 11  Description            object
 12  Number                 float64
 13  Street                 object
 14  Side                   object
 15  City                   object
 16  County                 object
 17  State                  object
 18  Zipcode                object
 19  Country                object
 20  Timezone               object
 21  Airport_Code           object
 22  Weather_Timestamp      object
 23  T

We will also check the number of missing values in the dataset, so that we can drop or fill in the missing values.

In [6]:
df.isna().sum()

ID                             0
Source                         0
TMC                      1034799
Severity                       0
Start_Time                     0
End_Time                       0
Start_Lat                      0
Start_Lng                      0
End_Lat                  2478818
End_Lng                  2478818
Distance(mi)                   0
Description                    1
Number                   2262864
Street                         0
Side                           0
City                         112
County                         0
State                          0
Zipcode                     1069
Country                        0
Timezone                    3880
Airport_Code                6758
Weather_Timestamp          43323
Temperature(F)             65732
Wind_Chill(F)            1868249
Humidity(%)                69687
Pressure(in)               55882
Visibility(mi)             75856
Wind_Direction             58874
Wind_Speed(mph)           454609
Precipitat

There are many columns with null values, and we will fill them with random values or the mean from the column. We will drop some text columns, as we are not doing any natural language processing right now, but feel free to explore them on your own. We will also drop the columns with too many Nans as filling them will throw our accuracy.

In [7]:
df = df.drop(columns = ['ID','Start_Time','End_Time','Street','Side','Description','Number','City','Country','Zipcode','Timezone','Airport_Code','Weather_Timestamp','Wind_Chill(F)','Wind_Direction','Wind_Speed(mph)','Precipitation(in)'])

In [8]:
#Here we are filling the TMC with mean.
df['TMC'] = df['TMC'].fillna(df['TMC'].mean())
df['End_Lat'] = df['End_Lat'].fillna(df['End_Lat'].mean())
df['End_Lng'] = df['End_Lng'].fillna(df['End_Lng'].mean())
df['Temperature(F)'] = df['Temperature(F)'].fillna(df['Temperature(F)'].mean())
df['Humidity(%)'] = df['Humidity(%)'].fillna(df['Humidity(%)'].mean())
df['Pressure(in)'] = df['Pressure(in)'].fillna(df['Pressure(in)'].mean())
df['Visibility(mi)'] = df['Visibility(mi)'].fillna(df['Visibility(mi)'].mean())
df['Humidity(%)'] = df['Humidity(%)'].fillna(df['Humidity(%)'].mean())
df['Pressure(in)'] = df['Pressure(in)'].fillna(df['Pressure(in)'].mean())
df['Visibility(mi)'] = df['Visibility(mi)'].fillna(df['Visibility(mi)'].mean())


df['Weather_Condition'] = df['Weather_Condition'].fillna('Fair')
df['Sunrise_Sunset'] = df['Sunrise_Sunset'].fillna('Day')
df['Civil_Twilight'] = df['Civil_Twilight'].fillna('Day')
df['Nautical_Twilight'] = df['Nautical_Twilight'].fillna('Day')
df['Astronomical_Twilight'] = df['Astronomical_Twilight'].fillna('Day')
df['Weather_Condition'] = df['Weather_Condition'].fillna('Fair')

Now all the columns contain no Nan values and we can go ahead with the preprocessing.

<a id='ex2'></a>
       
As you have observed in the dataset we have the start and end coordinates,so  let us apply Haversine distance formula to get the accident coverage distance. Take note of how these functions use the row-wise operations, something that we have learnt before. If you need help while creating the user defined functions refer to this [notebook](02-Intro_to_cuDF_UDFs.ipynb).

In [12]:
from math import cos, sin, asin, sqrt, pi, atan2
from numba import cuda

In [13]:
#Modify the code in this cell

def haversine_distance_kernel(Start_Lat, Start_Lng, End_Lat, End_Lng, out):
 
    for i, (x_1, y_1, x_2, y_2) in enumerate(zip(Start_Lat, Start_Lng, End_Lat, End_Lng)):
 
     #Perform the computations here and store the final value in out[i]
              
        out[i] = 

In [ ]:
#Modify the code in this cell

%%time
#Add the arguments to the apply_rows function for the haversine distance kernel
df = df.apply_rows()

Wow! The code segment that previously took  7 minutes to compute, now gets executed in less than a second! 

In [15]:
#Modify the code in this cell

def haversine_distance_kernel(Start_Lat, Start_Lng, End_Lat, End_Lng, out):
 
    for i, (x_1, y_1, x_2, y_2) in enumerate(zip(Start_Lat, Start_Lng, End_Lat, End_Lng)):
 
 #Perform the computations here and store the final value in out[i]
        
        out[i] = 

In [ ]:
#Modify the code in this cell

%%time
#Add the arguments to the apply_chunks function for the haversine distance kernel
outdf = df.apply_chunks()

Save the dataframe in a csv for future use, and make sure you refer to our sample solution and compared your code's performance with it.

In [17]:
df.head()

,Source,TMC,Severity,Start_Lat,Start_Lng,End_Lat,End_Lng,Distance(mi),Temperature(F),Humidity(%),...,WC_Thunderstorm,WC_Thunderstorms and Rain,WC_Thunderstorms and Snow,WC_Tornado,WC_Volcanic Ash,WC_Widespread Dust,WC_Widespread Dust / Windy,WC_Wintry Mix,WC_Wintry Mix / Windy,out
0,MapQuest,201.0,3,39.865147,-84.058723,37.557578,-100.455981,0.01,36.9,91.0,...,0,0,0,0,0,0,0,0,0,1443.524390
1,MapQuest,201.0,2,39.928059,-82.831184,37.557578,-100.455981,0.01,37.9,100.0,...,0,0,0,0,0,0,0,0,0,1548.467903
2,MapQuest,201.0,2,39.063148,-84.032608,37.557578,-100.455981,0.01,36.0,100.0,...,0,0,0,0,0,0,0,0,0,1440.697621
3,MapQuest,201.0,3,39.747753,-84.205582,37.557578,-100.455981,0.01,35.1,96.0,...,0,0,0,0,0,0,0,0,0,1429.927497
4,MapQuest,201.0,2,39.627781,-84.188354,37.557578,-100.455981,0.01,36.0,89.0,...,0,0,0,0,0,0,0,0,0,1430.383177


In [ ]:
df = df.dropna()

In [ ]:
df.to_csv("../../data/data_proc.csv")

# Conclusion 

Thus we have successfully used CuDF and CuPy to process the accidents dataset, and converted the data to a form more suitable to apply machine learning algorithms. In the extra labs for future labs in CuML we will be using this processed dataset. You must have observed the parallels between the RAPIDS pipeline and traditional pipeline while writing your code. Try to experiment with the processing and making your code as efficient as possible.

# References

- Moosavi, Sobhan, Mohammad Hossein Samavatian, Srinivasan Parthasarathy, and Rajiv Ramnath. “A Countrywide Traffic Accident Dataset.”, 2019.

- Moosavi, Sobhan, Mohammad Hossein Samavatian, Srinivasan Parthasarathy, Radu Teodorescu, and Rajiv Ramnath. "Accident Risk Prediction based on Heterogeneous Sparse Data: New Dataset and Insights." In proceedings of the 27th ACM SIGSPATIAL International Conference on Advances in Geographic Information Systems, ACM, 2019.

- If you need to refer to the dataset, you can download it [here](https://www.kaggle.com/sobhanmoosavi/us-accidents).

<center><a rel="license" href="http://creativecommons.org/licenses/by-nc-sa/4.0/"><img alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by-nc-sa/4.0/88x31.png" /></a></center><br />

- This dataset is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by-nc-sa/4.0/">Creative Commons Attribution-NonCommercial-ShareAlike 4.0 International License</a>.

## Licensing
  
This material is released by NVIDIA Corporation under the Creative Commons Attribution 4.0 International (CC BY 4.0).

[Previous Notebook](03-Cudf_Exercise.ipynb)
&emsp;&emsp;&emsp;&emsp;&emsp;
&emsp;&emsp;&emsp;&emsp;&emsp;
&emsp;&emsp;&emsp;&emsp;&emsp;
&emsp;&emsp;&emsp;&emsp;&emsp;
[1](01-Intro_to_cuDF.ipynb)
[2](02-Intro_to_cuDF_UDFs.ipynb)
[3](03-Cudf_Exercise.ipynb)
[4]
&emsp;&emsp;&emsp;&emsp;&emsp;
&emsp;&emsp;&emsp;&emsp;&emsp;
&emsp;&emsp;&emsp;&emsp;&emsp;
&emsp;&emsp;&emsp;&emsp;&emsp;


&emsp;&emsp;&emsp;&emsp;&emsp;
&emsp;&emsp;&emsp;&emsp;&emsp;
&emsp;&emsp;&emsp;&emsp;&emsp;
&emsp;&emsp;&emsp;&emsp;&emsp;
&emsp;&emsp;&emsp;&emsp;&emsp;
&emsp;&emsp;&ensp;
[Home Page](../START_HERE.ipynb)